# Phase 3: Advanced - Quantitative Analysis & Alpha Generation

In this phase, we implement a **Quantitative Refinement Loop**. This is a standard professional workflow where agents collaborate to profile data, propose predictive signals (alphas), verify the logic, and finally backtest the performance.

### System Architecture
We use a linear refinement pipeline:
```
Data Profiler → Alpha Builder → Signal Verifier → Backtester Agent → Performance Report
```

| Agent | Role |
|---|---|
| **DataProfiler** | Extracts statistical features (volatility, skew, kurtosis) |
| **AlphaBuilder** | Proposes a mathematical trading signal |
| **SignalVerifier** | Checks for logic errors (overfitting, bias) |
| **BacktesterAgent** | Simulates performance and calculates Sharpe Ratio |

## 1. Environment Setup

In [1]:
%pip install vinagent==0.0.6.post3

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv('.env'))

llm = ChatOpenAI(model="gpt-4o-mini")
print("LLM initialized.")

Note: you may need to restart the kernel to use updated packages.
LLM initialized.


## 2. Define Quant State

The `QuantState` tracks the progression from raw data to a verified backtest.

In [2]:
import operator
from typing import Annotated, List, TypedDict

def append_messages(existing: list, update: dict) -> list:
    return existing + [update]

class QuantState(TypedDict):
    """Coherent state for the quantitative refinement loop."""
    # List reducers for automatic merging
    messages: Annotated[list[dict], append_messages]
    data_profile: str
    alpha_signal: str
    verification_report: str
    backtest_metrics: str
    final_report: str

In [3]:
from datetime import datetime
from vinagent.register import primary_function

@primary_function
def get_current_time() -> str:
    """
    Get the current date and time. Use this to know 'today's' date.
    Returns:
        str: Current date and time in YYYY-MM-DD HH:MM:SS format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Custom time tool defined.")

Custom time tool defined.


## 3. Implement Quantitative Agents

Each agent adds scientific rigor to the signal generation process.

In [4]:
from vinagent.multi_agent import AgentNode
from vinagent.logger.logger import logging_message

class DataProfiler(AgentNode):
    """Analyzes statistical properties of the asset."""
    @logging_message
    def exec(self, state: QuantState) -> dict:
        print(f"[{self.name}] Analyzing market data and profiling distributions...")
        symbol = state.get("symbol", "FPT")
        prompt = f"""
        Calculate statistical metrics for {symbol} over the last 2 years.
        Use the `calculate_stock_statistics` tool.
        Explain the implications of volatility, skewness, and drawdown.
        """
        output = self.invoke(prompt, max_iterations=5)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "data_profile": output.content if hasattr(output, "content") else str(output)
        }

class AlphaBuilder(AgentNode):
    """Proposes a mathematical signal based on the data profile."""
    @logging_message
    def exec(self, state: QuantState) -> dict:
        print(f"[{self.name}] Proposing trading alpha based on profile...")
        profile = state.get("data_profile", "")
        prompt = f"""
        Based on this data profile: \n{profile}\n\n
        Propose a specific trading strategy ('momentum' or 'mean_reversion') and parameters (e.g., window size).
        Explain why it fits the profile. Format your choice clearly for the next agent.
        """
        output = self.invoke(prompt, max_iterations=3)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "alpha_signal": output.content if hasattr(output, "content") else str(output)
        }

class SignalVerifier(AgentNode):
    """Challenges the logic to prevent overfitting."""
    @logging_message
    def exec(self, state: QuantState) -> dict:
        print(f"[{self.name}] Verifying alpha logic...")
        signal = state.get("alpha_signal", "")
        prompt = f"Validate this alpha signal for common hedge fund pitfalls: \n{signal}\n\nProvide a verification report and approve or suggest adjustments."
        output = self.invoke(prompt, max_iterations=3)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "verification_report": output.content if hasattr(output, "content") else str(output)
        }

class BacktesterAgent(AgentNode):
    """Simulates the signal and provides Sharpe/Drawdown metrics."""
    @logging_message
    def exec(self, state: QuantState) -> dict:
        print(f"[{self.name}] Running backtest simulation...")
        symbol = state.get("symbol", "FPT")
        signal = state.get("alpha_signal", "")
        prompt = f"""
        Simulate the strategy described here: \n{signal}\n\n

        Identify the symbol ({symbol}), strategy_type (momentum/mean_reversion), and parameters.
        Use the `backtest_alpha_strategy` tool to get metrics.
        
        CRITICAL JSON RULE: If passing arguments to `kwargs`, format it strictly as JSON using double quotes. 
        Example: {{"window_size": 20}}
        DO NOT use single quotes.
        
        Provide a final performance report in Markdown.
        """
        output = self.invoke(prompt, max_iterations=5)
        return {
            "messages": [{"role": "assistant", "content": output.content if hasattr(output, "content") else str(output)}],
            "backtest_metrics": output.content if hasattr(output, "content") else str(output)
        }


## 4. Assemble the Refinement Pipeline

We use a linear sequential flow to ensure each agent builds upon the previous's work.

In [5]:
# Tools imported directly
from customize_tools import (
    fetch_stock_data_vn as fetch_stock_data, 
    visualize_stock_data_vn as visualize_stock_data, 
    plot_returns_vn as plot_returns,
    calculate_stock_statistics,
    backtest_alpha_strategy
)
from vinagent.tools.websearch_tools import search_api

@primary_function
def get_current_time() -> str:
    """Get current system time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Core Infrastructure Initialized (Direct Registration).")


Core Infrastructure Initialized (Direct Registration).


In [6]:
from vinagent.multi_agent import CrewAgent
from vinagent.graph.operator import FlowStateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from vinagent.register import primary_function

# 1. Instantiate the Agents
instr = 'CRITICAL: Format tool arguments as strictly valid JSON. You MUST use double quotes (") for all property names and string values. NEVER use single quotes (\').'
no_tool_instr = instr + " IMPORTANT: DO NOT use any tools. Your only job is to reason and output text."

profiler = DataProfiler(name="profiler", llm=llm, instruction=instr)
builder = AlphaBuilder(name="builder", llm=llm, instruction=no_tool_instr)
verifier = SignalVerifier(name="verifier", llm=llm, instruction=no_tool_instr)
backtester = BacktesterAgent(name="backtester", llm=llm, instruction=instr)

all_tools = [
    primary_function(fetch_stock_data), 
    primary_function(visualize_stock_data), 
    primary_function(calculate_stock_statistics),
    get_current_time,
    primary_function(backtest_alpha_strategy)
]

for agent in [profiler, builder, verifier, backtester]:
    for tool in all_tools:
        agent.tools_manager.register_function_tool(tool)

print("Agents instantiated and tools registered with safety net.")

# 3. Assemble the Linear Pipeline (Crew)
crew = CrewAgent(
    llm=llm,
    checkpoint=MemorySaver(),
    graph=FlowStateGraph(QuantState),
    flow=[
        START >> profiler,
        profiler >> builder,
        builder >> verifier,
        verifier >> backtester,
        backtester >> END
    ]
)
print("Phase 3 Quant Pipeline Assembled successfully.")

INFO:vinagent.register.tool:Registered tool: fetch_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: visualize_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_stock_statistics (runtime)
INFO:vinagent.register.tool:Registered tool: get_current_time (runtime)
INFO:vinagent.register.tool:Registered tool: backtest_alpha_strategy (runtime)
INFO:vinagent.register.tool:Registered tool: fetch_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: visualize_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_stock_statistics (runtime)
INFO:vinagent.register.tool:Registered tool: get_current_time (runtime)
INFO:vinagent.register.tool:Registered tool: backtest_alpha_strategy (runtime)
INFO:vinagent.register.tool:Registered tool: fetch_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: visualize_stock_data_vn (runtime)
INFO:vinagent.register.tool:Registered tool: calculate_stock_statistics 

Agents instantiated and tools registered with safety net.
Phase 3 Quant Pipeline Assembled successfully.


## 5. Run the Search for Alpha

Let's target AMZN for signal discovery.

In [8]:
initial_state = {"symbol": "FPT"}
result = crew.invoke(query="Find a momentum-based alpha for FPT.", user_id='admin', thread_id=67)

print("\n--- FINAL QUANT REPORT ---\n")
print(result.get("backtest_metrics", "No metrics found in state."))


INFO:vinagent.multi_agent.crew:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:vinagent.agent.agent:Tool calling iteration 1/5


{'input': {'messages': {'role': 'user', 'content': 'Find a momentum-based alpha for FPT.'}}, 'config': {'configurable': {'user_id': 'admin'}, 'thread_id': 67}}
[profiler] Analyzing market data and profiling distributions...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 1 iterations.
INFO:vinagent.logger.logger:

============ DataProfiler Response ============
{'messages': [{'role': 'assistant', 'content': "The statistical metrics for FPT over the last two years are as follows:\n\n- **Annualized Volatility:** 0.2679\n- **Skewness:** 0.0093\n- **Kurtosis:** 3.8633\n- **Average Daily Return:** 0.000624\n- **Sharpe Ratio:** 0.59\n- **Max Drawdown:** -0.3075\n\n**Implications:**\n\n1. **Volatility:** A higher volatility indicates greater price fluctuations, which can signal higher investment risk but also the potential for higher returns.\n  \n2. **Skewness:** A skewness near zero suggests that the returns are symmetrically distributed. Positive skewness indicates potential for higher than average gains, while negative skewness suggests risks of larger losses.\n\n3. **Drawdown:** The maximum drawdown 

[builder] Proposing trading alpha based on profile...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'backtest_alpha_strategy', 'tool_type': 'function', 'arguments': {'symbol': 'FPT', 'strategy_type': 'mean_reversion', 'start_date': '2021-10-01', 'end_date': '2023-10-01', 'kwargs': {'window_size': 10}}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool backtest_alpha_strategy({'symbol': 'FPT', 'strategy_type': 'mean_reversion', 'start_date': '2021-10-01', 'end_date': '2023-10-01', 'kwargs': {'window_size': 10}})
INFO:vinagent.agent.agent:Tool calling iteration 2/3
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ AlphaBuilder Response ============
{'messages': [{'role': 'assistant', 'content': 'Based on the provided data profile for 

[verifier] Verifying alpha logic...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 1 iterations.
INFO:vinagent.logger.logger:

============ SignalVerifier Response ============
{'messages': [{'role': 'assistant', 'content': "The recommendation for a mean reversion strategy with a 10-day window appears to be logical given the provided data profile for FPT. However, the following points need to be considered for a comprehensive evaluation:\n\n1. **Volatility**: The volatility figure of 0.2679 indicates a relatively volatile stock, which indeed supports the mean reversion strategy. However, it is essential to ensure that the volatility does not lead to excessive whipsaw losses.\n\n2. **Skewness**: The skewness of 0.0093 indicates a nearly symmetrical distribution of returns, suggesting that mean reversion could effectively harness price fluctuations in both upward and downward movements.\n\n3. **Max Drawdown**: The 

[backtester] Running backtest simulation...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'backtest_alpha_strategy', 'tool_type': 'function', 'arguments': {'symbol': 'FPT', 'strategy_type': 'mean_reversion', 'start_date': '2020-01-01', 'end_date': '2026-02-28', 'kwargs': '{"window_size": 10}'}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool backtest_alpha_strategy({'symbol': 'FPT', 'strategy_type': 'mean_reversion', 'start_date': '2020-01-01', 'end_date': '2026-02-28', 'kwargs': '{"window_size": 10}'})
INFO:vinagent.agent.agent:Tool calling iteration 2/5
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ BacktesterAgent Response ============
{'messages': [{'role': 'assistant', 'content': '### Performance Report for Mean 


--- FINAL QUANT REPORT ---

### Performance Report for Mean Reversion Strategy (Window Size: 10 Days)

- **Strategy**: Mean Reversion
- **Total Return**: -61.22%
- **Annualized Return**: -13.8%
- **Sharpe Ratio**: -0.57
- **Max Drawdown**: -64.31%

This performance report reflects the results of implementing a mean reversion trading strategy for the FPT stock based on the specified metrics and time frame.
